In [1]:
rm(list = ls())
gc();
gc();
library(data.table)
library(e1071)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,525820,28.1,1188290,63.5,621911,33.3
Vcells,954331,7.3,8388608,64.0,1602072,12.3


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,533174,28.5,1188290,63.5,621911,33.3
Vcells,970307,7.5,8388608,64.0,1602072,12.3


Warning message:
"package 'data.table' was built under R version 3.6.2"Warning message:
"package 'e1071' was built under R version 3.6.2"

In [2]:
options(repr.matrix.max.cols=100, repr.matrix.max.rows=100)

In [3]:
'%nin%' = Negate('%in%')

In [4]:
train = fread("D:/data_science/data/house-prices-advanced-regression-techniques/train.csv",sep = ",",
              colClasses = "character",na.strings = c(NA,'NULL','null','N/A','',' '))

Warning message in fread("D:/data_science/data/house-prices-advanced-regression-techniques/train.csv", :
"na.strings[6]==" " consists only of whitespace, ignoring. strip.white==TRUE (default) and "" is present in na.strings, so any number of spaces in string columns will already be read as <NA>."

In [5]:
head(train)

Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NA,Attchd,2003,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NA,NA,NA,0,2,2008,WD,Normal,208500
2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NA,NA,NA,0,5,2007,WD,Normal,181500
3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NA,NA,NA,0,9,2008,WD,Normal,223500
4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NA,NA,NA,0,12,2008,WD,Normal,250000
6,50,RL,85,14115,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NA,Attchd,1993,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NA,MnPrv,Shed,700,10,2009,WD,Normal,143000


In [6]:
fact_vars = c("MSSubClass", "MSZoning",  "Street", 
              "Alley", "LotShape", "LandContour", "Utilities", "LotConfig", 
              "LandSlope", "Neighborhood", "Condition1", "Condition2", "BldgType", 
              "HouseStyle", "OverallQual", "OverallCond",
              "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", 
               "ExterQual", "ExterCond", "Foundation", "BsmtQual", 
              "BsmtCond", "BsmtExposure", "BsmtFinType1",  "BsmtFinType2", 
                "Heating", "HeatingQC", "Fireplaces",
              "CentralAir", "Electrical",  "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", 
              "BedroomAbvGr", "KitchenAbvGr", "KitchenQual",
              "Functional", "FireplaceQu", "GarageType", 
              "GarageFinish","GarageQual", "GarageCond", 
              "PavedDrive",  "PoolQC", "Fence", "MiscFeature", 
                "SaleType", "SaleCondition","MoSold")

In [7]:
date_field = c( "YearBuilt", "YearRemodAdd","GarageYrBlt", "YrSold")

In [8]:
num_vars = c("LotFrontage","LotArea","MasVnrArea", "BsmtFinSF1","BsmtFinSF2","BsmtUnfSF",
             "TotalBsmtSF","1stFlrSF", "2ndFlrSF", "LowQualFinSF", 
             "GrLivArea","TotRmsAbvGrd","GarageCars","GarageArea",
             "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", 
             "ScreenPorch", "PoolArea","MiscVal")

In [9]:
target = "SalePrice"

In [10]:
train = train[, (num_vars) := lapply(.SD, as.numeric), .SDcols = num_vars]

In [11]:
skew_dt = as.data.frame(as.numeric(sapply(train[,.SD,.SDcols = num_vars], skewness)))
names(skew_dt) = c('skewness')
skew_dt$num_vars = num_vars
skew_dt  = as.data.table(skew_dt)
skew_dt = skew_dt[,.(num_vars,skewness)]
skew_dt$skewness[1] = skewness(train$LotFrontage,na.rm = T)
skew_dt$skewness[3] = skewness(train$MasVnrArea,na.rm = T)

In [12]:
mean_dt = as.data.frame(sapply(train[,.SD,.SDcols = num_vars], mean))
names(mean_dt) = c('mean')
mean_dt$num_vars = num_vars
mean_dt  = as.data.table(mean_dt)
mean_dt = mean_dt[,.(num_vars,mean)]
mean_dt$mean[1] = mean(train$LotFrontage,na.rm =T)
mean_dt$mean[3] = mean(train$MasVnrArea,na.rm =T)

In [13]:
median_dt = as.data.frame(sapply(train[,.SD,.SDcols = num_vars], median))
names(median_dt) = c('median')
median_dt$num_vars = num_vars
median_dt  = as.data.table(median_dt)
median_dt = median_dt[,.(num_vars,median)]
median_dt$median[1] = median(train$LotFrontage,na.rm =T)
median_dt$median[3] = median(train$MasVnrArea,na.rm =T)

In [14]:
sd_dt = as.data.frame(sapply(train[,.SD,.SDcols = num_vars], sd))
names(sd_dt) = c('sd')
sd_dt$num_vars = num_vars
sd_dt  = as.data.table(sd_dt)
sd_dt = sd_dt[,.(num_vars,sd)]
sd_dt$sd[1] = sd(train$LotFrontage,na.rm =T)
sd_dt$sd[3] = sd(train$MasVnrArea,na.rm =T)

In [15]:
na_count_dt = as.data.frame(sapply(train[,.SD,.SDcols = c(fact_vars,num_vars,date_field)], function(x) sum(is.na(x))))
names(na_count_dt) = c('na_count')
na_count_dt$vars = c(fact_vars,num_vars,date_field)
na_count_dt  = as.data.table(na_count_dt)
na_count_dt = na_count_dt[,.(vars,na_count)]
na_count_dt = na_count_dt[,na_perc := na_count/dim(train)[1]*100]
na_count_dt = na_count_dt[,to_drop := ifelse((na_count/dim(train)[1]*100>=30),1,0)]

In [16]:
class_desc = c(rep('factor',length(fact_vars)),rep('numeric',length(num_vars)),rep('date',length(date_field)))
na_count_dt = na_count_dt[,class_desc:= class_desc]

In [17]:
all_var_one_view = na_count_dt

setkey(all_var_one_view,vars)
setkey(mean_dt,num_vars)

In [18]:
all_var_one_view = merge(all_var_one_view,mean_dt,all.x = T,by.x = "vars",by.y = "num_vars")
setkey(median_dt,num_vars)

In [19]:
all_var_one_view = merge(all_var_one_view,median_dt,all.x = T,by.x = "vars",by.y = "num_vars")
setkey(sd_dt,num_vars)

In [20]:
all_var_one_view = merge(all_var_one_view,sd_dt,all.x = T,by.x = "vars",by.y = "num_vars")

In [21]:
setkey(skew_dt,num_vars)
all_var_one_view = merge(all_var_one_view,skew_dt,all.x = T,by.x = "vars",by.y = "num_vars")

In [22]:
for(i in 1:dim(all_var_one_view)[1])
{
  if(all_var_one_view$class_desc[i]=="factor" & (all_var_one_view$na_perc[i]<=30) && (all_var_one_view$na_perc[i]>0))
  {
    print("cleaning factor variables")
    print(all_var_one_view$vars[i])
    req_col = as.vector(train[,.SD,.SDcols = all_var_one_view$vars[i]])
    ind = which(is.na(req_col))
    req_col[ind] = "MISSING"
    train = train[,all_var_one_view$vars[i] := req_col]
  }
}

[1] "cleaning factor variables"
[1] "BsmtCond"
[1] "cleaning factor variables"
[1] "BsmtExposure"
[1] "cleaning factor variables"
[1] "BsmtFinType1"
[1] "cleaning factor variables"
[1] "BsmtFinType2"
[1] "cleaning factor variables"
[1] "BsmtQual"
[1] "cleaning factor variables"
[1] "Electrical"
[1] "cleaning factor variables"
[1] "GarageCond"
[1] "cleaning factor variables"
[1] "GarageFinish"
[1] "cleaning factor variables"
[1] "GarageQual"
[1] "cleaning factor variables"
[1] "GarageType"
[1] "cleaning factor variables"
[1] "MasVnrType"


In [23]:
for(i in 1:dim(all_var_one_view)[1])
{
  if(all_var_one_view$class_desc[i]=="numeric" & (all_var_one_view$na_perc[i]<=30) && (all_var_one_view$na_perc[i]>0))
  {
    print("cleaning numeric variables")
    print(all_var_one_view$vars[i])
    req_col1 = as.vector(train[,.SD,.SDcols = all_var_one_view$vars[i]])
    ind = which(is.na(req_col1))
    median_val = all_var_one_view$median[i]
    req_col1[ind] = median_val
    train = train[,all_var_one_view$vars[i] := req_col1]
  }
}

[1] "cleaning numeric variables"
[1] "LotFrontage"
[1] "cleaning numeric variables"
[1] "MasVnrArea"


In [24]:
all_var_one_view = all_var_one_view[,entropy_value := NA]
for(i in 1:dim(all_var_one_view)[1])
{
  if(all_var_one_view$class_desc[i] %in% c("factor","date"))
  {
    print("computing entropy for factor and date variables")
    print(all_var_one_view$vars[i])
    req_col = as.vector(train[,.SD,.SDcols = all_var_one_view$vars[i]])
    ent1 = as.vector(prop.table(table(req_col)))
    ent2 = (-1)*log(ent1)
    ent3 = ent1*ent2
    ent4 = sum(ent3)
    all_var_one_view$entropy_value[i] = ent4
  }
}

[1] "cleaning factor variables"
[1] "Alley"
[1] "cleaning factor variables"
[1] "BedroomAbvGr"
[1] "cleaning factor variables"
[1] "BldgType"
[1] "cleaning factor variables"
[1] "BsmtCond"
[1] "cleaning factor variables"
[1] "BsmtExposure"
[1] "cleaning factor variables"
[1] "BsmtFinType1"
[1] "cleaning factor variables"
[1] "BsmtFinType2"
[1] "cleaning factor variables"
[1] "BsmtFullBath"
[1] "cleaning factor variables"
[1] "BsmtHalfBath"
[1] "cleaning factor variables"
[1] "BsmtQual"
[1] "cleaning factor variables"
[1] "CentralAir"
[1] "cleaning factor variables"
[1] "Condition1"
[1] "cleaning factor variables"
[1] "Condition2"
[1] "cleaning factor variables"
[1] "Electrical"
[1] "cleaning factor variables"
[1] "ExterCond"
[1] "cleaning factor variables"
[1] "ExterQual"
[1] "cleaning factor variables"
[1] "Exterior1st"
[1] "cleaning factor variables"
[1] "Exterior2nd"
[1] "cleaning factor variables"
[1] "Fence"
[1] "cleaning factor variables"
[1] "FireplaceQu"
[1] "cleaning factor va

In [25]:
train = train[,LotFrontage_c := ifelse(LotFrontage <=59,'<=59', ifelse(LotFrontage<=69,'<=69',ifelse(LotFrontage<=80,'<=80','>80')))]
train = train[,LotArea_c := ifelse(LotArea <=7553.5,'<=7553.5', ifelse(LotArea<=9478.5,'<=9478.5',ifelse(LotArea<=11601.5,'<=11601.5','>11601.5')))]
train = train[,MasVnrArea_c := ifelse(MasVnrArea<=166,'<=166','>166')]
train = train[,BsmtFinSF1_c := ifelse(BsmtFinSF1<=383.5,'<=383.5',ifelse(BsmtFinSF1<=712.25,'<=712.25','>712.25'))]

train = train[,BsmtUnfSF_c := ifelse(BsmtUnfSF <=223,'<=223', ifelse(BsmtUnfSF<=477.5,'<=477.5',ifelse(BsmtUnfSF<=808,'<=808','>808')))]
train = train[,TotalBsmtSF_c := ifelse(TotalBsmtSF <=795.75,'<=795.75', ifelse(TotalBsmtSF<=991.5,'<=991.5',ifelse(TotalBsmtSF<=1298.25,'<=1298.25','>1298.25')))]
train = train[,FirstFlrSF_c := ifelse(`1stFlrSF` <=882,'<=882', ifelse(`1stFlrSF`<=1087,'<=1087',ifelse(`1stFlrSF`<=1391.25,'<=1391.25','>1391.25')))]

train = train[,GrLivArea_c := ifelse(GrLivArea <=1129.5,'<=1129.5', ifelse(GrLivArea<=1464,'<=1464',ifelse(GrLivArea<=1776.75,'<=1776.75','>1776.75')))]
train = train[,TotRmsAbvGrd_c := ifelse(TotRmsAbvGrd <=5,'<=5', ifelse(TotRmsAbvGrd<=6,'<=6',ifelse(TotRmsAbvGrd<=7,'<=7','>7')))]
train = train[,GarageCars_c := ifelse(GarageCars <=1,'<=1', ifelse(GarageCars<=2,'<=2',ifelse(GarageCars<=2,'<=2','>2')))]
train = train[,GarageArea_c := ifelse(GarageArea <=334.5,'<=334.5', ifelse(GarageArea<=480,'<=480',ifelse(GarageArea<=576,'<=576','>576')))]
train = train[,WoodDeckSF_c := ifelse(WoodDeckSF<=168,'<=168','>168')]
train = train[,OpenPorchSF_c := ifelse(OpenPorchSF<=25,'<=25',ifelse(OpenPorchSF<=68,'<=68','>68'))]

In [26]:
for(i in 1:dim(all_var_one_view)[1])
{
  if(all_var_one_view$class_desc[i]=="numeric" & all_var_one_view$vars[i] %nin% c('1stFlrSF','2ndFlrSF','3SsnPorch',
                                                                                  'BsmtFinSF2','EnclosedPorch','LowQualFinSF',
                                                                                  'MiscVal','PoolArea','ScreenPorch'))
  {
    print("computing entropy for numerical variable")
    print(all_var_one_view$vars[i])
    req_col_name = paste0(all_var_one_view$vars[i],"_c")
    req_col = as.vector(train[,.SD,.SDcols = req_col_name])
    ent1 = as.vector(prop.table(table(req_col)))
    ent2 = (-1)*log(ent1)
    ent3 = ent1*ent2
    ent4 = sum(ent3)
    all_var_one_view$entropy_value[i] = ent4
  }
}

[1] "computing entropy for numerical variable"
[1] "BsmtFinSF1"
[1] "computing entropy for numerical variable"
[1] "BsmtUnfSF"
[1] "computing entropy for numerical variable"
[1] "GarageArea"
[1] "computing entropy for numerical variable"
[1] "GarageCars"
[1] "computing entropy for numerical variable"
[1] "GrLivArea"
[1] "computing entropy for numerical variable"
[1] "LotArea"
[1] "computing entropy for numerical variable"
[1] "LotFrontage"
[1] "computing entropy for numerical variable"
[1] "MasVnrArea"
[1] "computing entropy for numerical variable"
[1] "OpenPorchSF"
[1] "computing entropy for numerical variable"
[1] "TotRmsAbvGrd"
[1] "computing entropy for numerical variable"
[1] "TotalBsmtSF"
[1] "computing entropy for numerical variable"
[1] "WoodDeckSF"


In [27]:
train = train[,house_vintage := as.numeric(YrSold)- as.numeric(YearBuilt)]
train = train[,garage_vintage := as.numeric(YrSold)- as.numeric(GarageYrBlt)]
train = train[,time_since_remodelling := as.numeric(YrSold) - as.numeric(YearRemodAdd)]

In [28]:
req_col1 = as.vector(train[,.SD,.SDcols = "garage_vintage"])
ind = which(is.na(req_col1))
median_val = median(train$garage_vintage,na.rm = T)
req_col1[ind] = median_val
train = train[,garage_vintage := req_col1]

In [29]:
train = train[, SalePrice := as.numeric(SalePrice)]
train = train[, SalePrice_log := log(SalePrice)]

mean_target = mean(train$SalePrice)
sd_target = sd(train$SalePrice)

train = train[,SalePrice_norm := (SalePrice-mean_target)/sd_target]

In [30]:
for(i in 1:dim(all_var_one_view)[1])
{
  if(all_var_one_view$class_desc[i]=="numeric")
  {
    req_col1 = as.vector(train[,.SD,.SDcols = all_var_one_view$vars[i]])
    ind = which(((req_col1- all_var_one_view$mean[i])/all_var_one_view$sd[i])>=3.5 | ((req_col1- all_var_one_view$mean[i])/all_var_one_view$sd[i])<= -3.5)
    median_val = all_var_one_view$median[i]
    if(length(ind)>0){
      print("analysing the outliers")
      print(all_var_one_view$vars[i])
    }
    req_col1[ind] = median_val
    train = train[,all_var_one_view$vars[i] := req_col1]
  }
}

[1] "analysing the outliers"
[1] "1stFlrSF"
[1] "analysing the outliers"
[1] "2ndFlrSF"
[1] "analysing the outliers"
[1] "3SsnPorch"
[1] "analysing the outliers"
[1] "BsmtFinSF1"
[1] "analysing the outliers"
[1] "BsmtFinSF2"
[1] "analysing the outliers"
[1] "BsmtUnfSF"
[1] "analysing the outliers"
[1] "EnclosedPorch"
[1] "analysing the outliers"
[1] "GarageArea"
[1] "analysing the outliers"
[1] "GrLivArea"
[1] "analysing the outliers"
[1] "LotArea"
[1] "analysing the outliers"
[1] "LotFrontage"
[1] "analysing the outliers"
[1] "LowQualFinSF"
[1] "analysing the outliers"
[1] "MasVnrArea"
[1] "analysing the outliers"
[1] "MiscVal"
[1] "analysing the outliers"
[1] "OpenPorchSF"
[1] "analysing the outliers"
[1] "PoolArea"
[1] "analysing the outliers"
[1] "ScreenPorch"
[1] "analysing the outliers"
[1] "TotRmsAbvGrd"
[1] "analysing the outliers"
[1] "TotalBsmtSF"
[1] "analysing the outliers"
[1] "WoodDeckSF"


In [31]:
to_drop = na_count_dt[to_drop==1,.(vars)]
col_to_drop = c()
for(i in 1:dim(to_drop)[1])
{
  col_to_drop[i] = as.character(to_drop[i])
}
cols_to_keep = names(train)[names(train) %nin% col_to_drop]

In [32]:
all_num_vars = c("LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", 
            "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", 
            "LowQualFinSF", "GrLivArea", "TotRmsAbvGrd", "GarageCars", "GarageArea", 
            "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", 
            "PoolArea", "MiscVal", "house_vintage","garage_vintage", "time_since_remodelling")

all_fact_vars = c("MSSubClass", "MSZoning", "Street", "Alley", "LotShape", "LandContour", 
                  "Utilities", "LotConfig", "LandSlope", "Neighborhood", "Condition1", 
                  "Condition2", "BldgType", "HouseStyle", "OverallQual", "OverallCond", 
                  "YearBuilt", "YearRemodAdd", "RoofStyle", "RoofMatl", "Exterior1st", 
                  "Exterior2nd", "MasVnrType", "ExterQual", "ExterCond", "Foundation", 
                  "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", 
                  "Heating", "HeatingQC", "CentralAir", "Electrical", "BsmtFullBath", 
                  "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr", 
                  "KitchenQual", "Functional", "Fireplaces", "FireplaceQu", "GarageType", 
                  "GarageYrBlt", "GarageFinish", "GarageQual", "GarageCond", "PavedDrive", 
                  "PoolQC", "Fence", "MiscFeature", "MoSold", "YrSold", "SaleType", 
                  "SaleCondition","LotFrontage_c", "LotArea_c", 
                  "MasVnrArea_c", "BsmtFinSF1_c", "BsmtUnfSF_c", "TotalBsmtSF_c", 
                  "FirstFlrSF_c", "GrLivArea_c", "TotRmsAbvGrd_c", "GarageCars_c", 
                  "GarageArea_c", "WoodDeckSF_c", "OpenPorchSF_c")

In [34]:
cor(train[,.SD,.SDcols = all_num_vars])

Warning message in cor(train[, .SD, .SDcols = all_num_vars]):
"the standard deviation is zero"

,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,TotRmsAbvGrd,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,house_vintage,garage_vintage,time_since_remodelling
LotFrontage,1.00000000,0.504984932,0.14458416,1.183670e-01,0.0166078026,0.144128569,0.314886818,0.369640394,0.045372151,-0.051487199,0.295648106,0.306421364,0.283562016,0.309231926,0.09840406,0.12252235,-0.0375579998,0.019678095,0.047702895,NA,1.106418e-02,-0.118300661,-0.063630505,-0.063498087
LotArea,0.50498493,1.000000000,0.08652706,1.772237e-01,0.0050004052,0.067949951,0.302094789,0.396346390,0.112662219,-0.032179935,0.346945806,0.343325432,0.278739350,0.298676857,0.16523401,0.13826140,-0.0312272740,-0.002124866,0.015566654,NA,1.251360e-02,-0.048350241,-0.004400542,-0.037794550
MasVnrArea,0.14458416,0.086527063,1.00000000,2.160260e-01,-0.0419610793,0.095580445,0.287771636,0.282506507,0.112392248,-0.040053639,0.311669792,0.232187414,0.334025793,0.330698507,0.14332837,0.17257537,-0.1037889686,-0.019298474,0.054022211,NA,-4.570131e-02,-0.296603110,-0.219372293,-0.149443209
BsmtFinSF1,0.11836699,0.177223731,0.21602599,1.000000e+00,0.0030142402,-0.523080957,0.440308844,0.368553487,-0.174882308,-0.038768656,0.122249301,-0.001911383,0.221861609,0.271926603,0.19854509,0.08194030,-0.1017794342,0.005880872,0.086360266,NA,4.012735e-06,-0.245245765,-0.138129913,-0.118261680
BsmtFinSF2,0.01660780,0.005000405,-0.04196108,3.014240e-03,1.0000000000,-0.177979268,-0.005226327,0.007350022,-0.078297654,-0.017718143,-0.049728853,-0.032927021,-0.064735163,-0.038575034,0.02304514,-0.06915441,0.0009713999,-0.008536880,0.016689060,NA,-1.173773e-02,0.069197491,0.086038577,0.095834907
BsmtUnfSF,0.14412857,0.067949951,0.09558045,-5.230810e-01,-0.1779792681,1.000000000,0.408048905,0.301244728,0.009554540,-0.009943328,0.234283730,0.242168658,0.203827172,0.169291166,-0.01501412,0.13939399,-0.0071103436,0.011452466,-0.031574782,NA,-4.613857e-02,-0.143703314,-0.181357974,-0.178403104
TotalBsmtSF,0.31488682,0.302094789,0.28777164,4.403088e-01,-0.0052263269,0.408048905,1.000000000,0.791240342,-0.213756535,-0.059790589,0.378091163,0.251658321,0.448344777,0.474091553,0.23258604,0.23828615,-0.1152114711,0.015019110,0.062100168,NA,-6.121119e-02,-0.399430655,-0.310910823,-0.294159067
1stFlrSF,0.36964039,0.396346390,0.28250651,3.685535e-01,0.0073500219,0.301244728,0.791240342,1.000000000,-0.239018780,-0.059950090,0.490696851,0.381715930,0.453648506,0.479505054,0.21315958,0.19462066,-0.1054999163,0.010565499,0.086943180,NA,-3.794534e-02,-0.281882871,-0.223539471,-0.244027233
2ndFlrSF,0.04537215,0.112662219,0.11239225,-1.748823e-01,-0.0782976542,0.009554540,-0.213756535,-0.239018780,1.000000000,0.022496499,0.659210622,0.608523147,0.179493830,0.139509687,0.07777771,0.19314288,0.0231545058,-0.025124832,-0.002738472,NA,-1.239579e-02,-0.008858562,-0.067965006,-0.140744733
LowQualFinSF,-0.05148720,-0.032179935,-0.04005364,-3.876866e-02,-0.0177181425,-0.009943328,-0.059790589,-0.059950090,0.022496499,1.000000000,-0.009086136,-0.039661599,-0.016475629,-0.024669165,-0.04099485,0.02769599,-0.0222610859,-0.002067810,-0.015447307,NA,6.747535e-02,0.051942597,0.010936033,-0.004182906


In [35]:
write.table(train,"D:/data_science/data/house-prices-advanced-regression-techniques/clean_train_IRkernel",sep = "^",row.names = FALSE,
            quote = FALSE)